In [2]:
import numpy as np

In [4]:
def compute_gray_code(dv):
    """
    returns the gray code
    If you begin with [0 for i in range(dv)] and flip the bits res[0], res[1], ..., res[dv-2] then you go through {0,1}^dv
    """
    res = []
    for i in range(dv):
        res = res + [i] + res
    return res

def hor_subset_score(hor_synd_diff, hor_wweight, ver_synd_diff, dv, dc):
    """
    'hor' means horizontal
    'ver' means vertical
    Input:
      'hor_synd_diff' is |s| - |s xor synd(F)| for the current F which is a flip of the columns
      'hor_wweight' is the weighted weight of the horizontal flips = dc * hor_flips
      ver_synd_diff[i] is the difference of syndrome size when we flip the line 'i'
    Output:
      'ver_flips' the optimal set of lines to flip for this given flips of columns
      'synd_diff' the syndrome difference for this flips
    When hor_weight = 0, i.e F = 0 then len(ver_flips) > 0 even if the flipping ver_flips increases the syndrome weight
    wweight = weighted weight = dv * len(ver_flips) + dc * len (hor_flips)
    """
    synd_diff = 0
    s = 0
    i = 0
    weight = 0
    len_ver_flips = 0

    synd_diff = hor_synd_diff
    # print(synd_diff, hor_synd_diff, ver_synd_diff)
    ver_flips = []
    sorted_ver_synd_diff = [(ver_synd_diff[i],i) for  i in range(len(ver_synd_diff))]
    sorted_ver_synd_diff.sort(reverse=True)

    wweight = hor_wweight
    for (s,i) in sorted_ver_synd_diff:
        if s*wweight >= dv * synd_diff:
            synd_diff = synd_diff + s
            ver_flips.append(i)
            wweight = wweight + dv

    return (synd_diff,ver_flips)

# @cython.binding(True)
def score_gen(synd_gen, synd_gen_mask):
    """
    Input:
      'synd_gen' is a 0,1 matrix which reprensents the syndrome of the current generator
      'gray_code' is the output of 'compute_gray_code'
    Output:
      'best_flips' = (ver_flips,hor_flips) are two lists of lines and columns which are optimal for the generator
      'best_synd_diff' is the syndrome difference for these flips
      'best_wweight' = dv * len(ver_flips) + dc * len(hor_flips)
    We go through all the possible flips of columns and use the function 'hor_subset_score'
    At the end, best_weight > 0 even it is better to flip nothing
    """
    i = 0
    j = 0
    dc = 0
    dv = 0

    hor_wweight = 0
    hor_synd_diff = 0

    best_wweight = 0
    best_synd_diff = 0

    dc = len(synd_gen)
    dv = len(synd_gen[0])
    hor_wweight = 0
    hor_flips_array = [False for j in range(dv)]
    hor_synd_diff = 0
    ver_synd_diff = [0 for i in range(dc)]
    for i in range(dc):
        for j in range(dv):
            if (synd_gen_mask[i][j]):
                ver_synd_diff[i] = ver_synd_diff[i] + 2*synd_gen[i][j] - 1

    gray_code = compute_gray_code(dv)
    (best_synd_diff,ver_flips) = hor_subset_score(hor_synd_diff, hor_wweight, ver_synd_diff, dv, dc)
    print(best_synd_diff, ver_flips)
    best_wweight = len(ver_flips) * dv
    best_flips = (ver_flips, [])
    for j in gray_code:
        if hor_flips_array[j]:
            hor_wweight = hor_wweight - dc
            hor_flips_array[j] = False
            for i in range(dc):
                if (synd_gen_mask[i][j]):
                    ver_synd_diff[i] = ver_synd_diff[i] + 4*synd_gen[i][j] - 2
                    hor_synd_diff = hor_synd_diff - 2*synd_gen[i][j] + 1
        else:
            hor_wweight = hor_wweight + dc
            hor_flips_array[j] = True
            for i in range(dc):
                if (synd_gen_mask[i][j]):
                    ver_synd_diff[i] = ver_synd_diff[i] - 4*synd_gen[i][j] + 2
                    hor_synd_diff = hor_synd_diff + 2*synd_gen[i][j] - 1

        (synd_diff,ver_flips) = hor_subset_score(hor_synd_diff, hor_wweight, ver_synd_diff, dv, dc)
        wweight = hor_wweight + dv * len(ver_flips)
        if synd_diff*best_wweight > best_synd_diff*wweight:
            best_synd_diff = synd_diff
            best_wweight = wweight
            best_flips = (ver_flips, [j for j in range(dv) if hor_flips_array[j]])

    return (best_synd_diff,best_wweight,best_flips)


In [6]:
synd_gen = np.array([[1],[0]])
synd_mask_gen = np.array([[1],[1]])

score_gen(synd_gen, synd_mask_gen)

1 [0]


(1, 1, ([0], []))